# Obtain data of AGN from SIMBAD and filtering the objects with data of Ne II, Ne III and Ne V in infrared from NED 

In this notebook we develop the code to Obtain the data of AGNs type Sy1 (Seyfert 1), Sy2 (Seyfert 2) and SyG (Seyfert Galaxies) from **[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** (SMB) and then search each object from the SMB sample in **[NED](https://ned.ipac.caltech.edu/)** to obtain the photometric data and filter all the objects from the sample that contain data of the emission lines of Ne II, Ne III and Ne V in infrared.

The part of the code to obtain the data from SIMBAD is based in Andres Ramos's work to obtain Seyfert Samples (https://github.com/aframosp/AGNView)

In [1]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [2]:
from datetime import date
import time
import csv
import numpy as np

import pyvo
from astroquery.ipac.ned import Ned
from astroquery.simbad import Simbad
from pathlib import Path

import astropy.units as u
from astropy.table import hstack, QTable
from astropy.coordinates import SkyCoord

In [3]:
path_raw='../Datos/Raw/'
Path(path_raw).mkdir(parents=True, exist_ok=True)
path_phot_ned = '../Datos/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
Path(path_phot_ned).mkdir(parents=True, exist_ok=True)
path_phot_cds = '../Datos/Phot_Tables_CDS/' #In this path wi will save the photometry tables for each object
Path(path_phot_cds).mkdir(parents=True, exist_ok=True)

Using the package **[PyVO](https://pyvo.readthedocs.io/en/latest/#)** we can access to remote data and services like the **TAP Service of SIMBAD [SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** to find and retrieve the astronomical data that we need, in this case: Seyfert Galaxies. To achive that, we use a script with **[ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language)**, which is based on SQL.

On the script we used the next **Field Names** for VOTable ouput:
- ra: Right ascension
- dec: Declination
- coo_bibcode: Bibliographical reference for coordinates
- otype_txt: Object type
- rvz_bibcode: Bibliographical reference of radial velocity and redshift
- rvz_radvel: Stored value. Either a radial velocity, or a redshift, depending on the rvz_type field 
- rvz_redshift
- rvz_type: Stored type of velocity: 'v'=radial velocity, 'z'=redshift
- nbref: bibliographical reference

~~~~sql
SELECT main_id
	,ra
	,dec
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

In [4]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

We save the date in a variable for use later in the file names of the results

In [5]:
today = date.today()
today.strftime('%b_%d_%Y')

'Apr_01_2022'

We extract the object name (or in this case the main id) from the sample result.

In [6]:
simbad_sample_Sy = result.to_table()
obj_name_SMB=simbad_sample_Sy["main_id"]
len(simbad_sample_Sy)

44272

In [7]:
INIT_URL = 'http://vizier.u-strasbg.fr/viz-bin/sed?-c='
urls = [INIT_URL+str(simbad_sample_Sy['ra'][row])+'%20'+
        str(simbad_sample_Sy['dec'][row])+'&-c.rs=5' for row in range(len(simbad_sample_Sy))]
simbad_sample_Sy['cds_url'] = urls

In [8]:
simbad_sample_Sy

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes,cds_url
,deg,deg,,,,km / s,,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object,str93
RX J1203.5+4432,180.88724599190996,2.4929432724600002,2020yCat.1350....0G,Sy1,2012ApJS..203...21A,22264.155457144552,0.07724,z,42,* |AGN|EmO|G |GiC|IR |QSO|Rad|Sy1|V* |X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=180.88724599190996%202.4929432724600002&-c.rs=5
2MASX J12014361+4307386,180.43169828928998,43.1273394439,2020yCat.1350....0G,Sy2,2009ApJS..182..543A,20748.633978793205,0.07178,z,4,* |G |Rad|Sy2|rG,http://vizier.u-strasbg.fr/viz-bin/sed?-c=180.43169828928998%2043.1273394439&-c.rs=5
SDSS J144452.42+511957.5,221.218431,51.332651,2012ApJS..203...21A,Sy2,2015ApJS..219...12A,25457.504881087,0.08885,z,4,G |GiG|Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=221.218431%2051.332651&-c.rs=5
FIRST J083344.2+283939,128.43421094374,28.66111602879,2020yCat.1350....0G,Sy1,2009ApJS..182..543A,173513.43668282355,0.936,z,13,* |AGN|QSO|Rad|Sy1,http://vizier.u-strasbg.fr/viz-bin/sed?-c=128.43421094374%2028.66111602879&-c.rs=5
SDSS J091658.99+012604.8,139.24580903083,1.43466726489,2020yCat.1350....0G,Sy1,2015ApJS..219...12A,125231.50632150602,0.56039,z,8,* |QSO|Sy1,http://vizier.u-strasbg.fr/viz-bin/sed?-c=139.24580903083%201.43466726489&-c.rs=5
MCG+06-34-028,235.23225416666668,32.78155555555555,2006AJ....131.1163S,Sy2,2002LEDA.........0P,15618.951821140114,0.05353,z,12,G |IR |Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=235.23225416666668%2032.78155555555555&-c.rs=5
IC 3834,192.88488958362,-14.221116888659996,2020yCat.1350....0G,Sy1,2002LEDA.........0P,4332.318180333962,0.014557,z,34,* |AGN|G |GiG|IR |Sy1,http://vizier.u-strasbg.fr/viz-bin/sed?-c=192.88488958362%20-14.221116888659996&-c.rs=5
SDSS J155011.46+233017.1,237.54777578073,23.50475679549,2020yCat.1350....0G,Sy2,2012ApJS..203...21A,33510.40645237621,0.11879,z,1,* |Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=237.54777578073%2023.50475679549&-c.rs=5


The sample results is saved in a .vot file:

In [9]:
simbad_sample_Sy.write(path_raw+'SMB_'+today.strftime('%b_%d_%Y')+'_Sy_Samples'+'.vot',
                    format='votable',overwrite=True)

In the next two cells, functions to download the Photometry tables from [NED](https://ned.ipac.caltech.edu/) and [CDS (Vizier)](http://vizier.unistra.fr/vizier/sed/?-c=&-c.r=10&-c.u=arcsec) are created. To get the photometry tables from NED, we use **[Astroquery](https://astroquery.readthedocs.io/en/latest/index.html)** package. In the function **photNED** first we try to obtain the photometry table with the ID name of the object from the SIMBAD sample. If there is not result, then with Astroquery we will search the others objects ID, and with that, search the information in NED. If finallly is not possible to find information with the objects ID, then we will try using its coordinates.

In [10]:
def photNED(name,coord):
    
    '''
    This function is to get the photometry tables
    from NED. The input are the id and coordinates
    of the object.
    '''
    
    phot_ned = QTable()
    id_used = []
    
    try:
        phot_ned = Ned.get_table(name, table='photometry') #We get the Photometry Table using first SIMBAD names
        id_used = name
    except:
        other_ids = Simbad.query_objectids(name) #We get other IDs for the object from SIMBAD
        for other_id in other_ids['ID']:
            try:
                phot_ned = Ned.get_table(other_id, table='photometry') #Photometry Table using other SIMBAD names
                id_used = other_id
                break
            except:
                pass
    if id_used == []:
        try:
            region = Ned.query_region(coord,radius= 5 * u.arcsec)
            id_ned = region["Object Name"]
            phot_ned = Ned.get_table(id_ned, table='photometry')
            id_used = id_ned
        except:
            pass

    return phot_ned, id_used

In [11]:
def photCDS(url,name,path):
    req = requests.get(url,
                       headers={'cache-control': 'private, max-age=0, no-cache', "Pragma": "no-cache"})
    if req.status_code == 200:
        with open(path+name+'_CDS_phot_tables.vot', 'wb') as f_vot:
            f_vot.write(req.content)
            f_vot.close()
    return req

Using **SkyCoord** from astropy we extract the coordinates information for each object from the SIMBAD samples. This is useful if we want to search information in [NED](https://ned.ipac.caltech.edu/) using coordinates instead the object name

In [12]:
cat_smb_sample = SkyCoord(ra=simbad_sample_Sy['ra'], dec=simbad_sample_Sy['dec'])

The wavelenght for NeII, NeIII and NeV in the infrared spectrum are 12.8 microns, 15.6 microns and 14.3 microns respectively. This values are equivalents to the following frequencies which ara related to different labels of "Observed Passband" in NED database as we ilustrate in the next list:

* Frequency: 1.92e+13 Hz: [NeIII] 15.6 microns
* Frecuency: 1.93e+13 Hz: [Ne III] 15.56 (IRS), [NeIII] 15.6 Spitzer, [Ne III] (Spitzer), [Ne III] 15.5 (IRS)
* Frecuency: 2.09e+13 Hz: [Ne V] 14.32 (IRS), [Ne V] 14.32 Spitzer, [Ne V] 14.32, [Ne V] 14.3 Spitzer, [Ne V] 14.3 microns
* Frequency: 2.10e+13 Hz: [Ne V] 14.3 (IRS)
* Frequency: 2.34e+13 Hz: [Ne II] 12.81 (IRS), [Ne II] 12.8 Spitzer, [Ne II] 12.8 (IRS), [Ne II] 12.81, NEII (VLT), 12.81 microns (VLT) 

In the variable **Ne_IR_Fq** we define a list with the those frequency values

In [13]:
Ne_IR_Fq = [19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0]

The filter for the objects with Ne Line Emissions in Infrared will be looking in the column **"Frequency"** of the photometry tables that we got from NED the values of the list defined in Ne_IR_Freq.

In [31]:
NeIR_obj_t = QTable(names=('Main_Id','Id_used_NED','RA','DEC','otype_txt','redshift',
                           'rvz_type','otypes','cds_url','SMB_sample_index'),
                    dtype=('O','O','f8','f8','O','f8','U1','O','U93','f8'))

print(f"{'#':4} {'Idx':6} {'Name ID':^27} {'ID used in NED':^27}")
a=0
start = time.time()
for ind, id_smb in enumerate(obj_name_SMB):    
    try:       
        #print(ind)
        phot_t,name_ned = photNED(id_smb,cat_smb_sample[ind])
        Fr=phot_t['Frequency'] #We get the Frequency column from the table    
        if Ne_IR_Fq[0] in Fr or Ne_IR_Fq[1] in Fr or Ne_IR_Fq[2] in Fr or Ne_IR_Fq[3] in Fr or Ne_IR_Fq[4] in Fr:
            a=a+1
            print(f"{'%g'%a:4} {'%g'%ind:6} {'%s'%id_smb:^27} {'%s'%name_ned:^27}")
            #Writing the object names
            
            NeIR_obj_row = [id_smb,name_ned,simbad_sample_Sy["ra"][ind]*u.deg,simbad_sample_Sy["dec"][ind]*u.deg,
                                   simbad_sample_Sy["otype_txt"][ind],simbad_sample_Sy["rvz_redshift"][ind],
                                   simbad_sample_Sy["rvz_type"][ind],simbad_sample_Sy["otypes"][ind],
                                   simbad_sample_Sy["cds_url"][ind],ind] 
            NeIR_obj_t.add_row(vals=NeIR_obj_row)
            
            phot_t.write(path_phot_ned+id_smb+'_NED_phot_tables'+'.vot', #Save the phot table in a vot table file
                         format='votable',overwrite=True)
    except KeyError:
        pass

end = time.time()

print('\nTotal objects with lines emission of NeII or NeIII or NeV in IR  found: ',len(Ne_IR_obj))
print('Execution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

#    Idx              Name ID                 ID used in NED       
1    283             3C 234.0                    3C 234.0          
2    414               M  84                       M  84           
3    428      2MASS J13000535+1632148       NVSS J130005+163212    
4    484             NGC   262                   NGC   262         
5    541          IRAS 04385-0828             IRAS 04385-0828      
6    628             NGC  4945                   NGC  4945         
7    654              IC 4553                     IC 4553          
8    773             NGC  5252                   NGC  5252         
9    852             Mrk  279                    Mrk  279          
10   1227            UGC 11680                   UGC 11680         
11   1246          LEDA   45656                LEDA   45656        
12   1282            Mrk 1239                    Mrk 1239          
13   1341          LEDA   89032                LEDA   89032        
14   1424          NAME 3C 330G2                

122  15091           PB  3894                    PB  3894          
123  15225             M  88                       M  88           
124  15263           NGC  4602                   NGC  4602         
125  15277        IRAS 13342+3932             IRAS 13342+3932      
126  15290           NGC  1241                   NGC  1241         
127  15365           NGC  4138                   NGC  4138         
128  15451           NGC  1566                   NGC  1566         
129  15630           NGC  1808                   NGC  1808         
130  15958     ICRF J054236.1+495107       ICRF J054236.1+495107   
131  16006           NGC  4074                   NGC  4074         
132  16060           Mrk  304                    Mrk  304          
133  16101           Mrk 1298                    Mrk 1298          
134  16220     2XMM J135315.8+634546       2XMM J135315.8+634546   
135  16350           Mrk 1095                    Mrk 1095          
136  16437        IRAS 15206+3342             IR

243  30302           NGC  6221                   NGC  6221         
244  30425          ESO  12-21                  ESO  12-21         
245  30923           UGC  5101                   UGC  5101         
246  31018          ESO 602-25                  ESO 602-25         
247  31077          ESO 511-30                  ESO 511-30         
248  31259         MCG-03-34-064               MCG-03-34-064       
249  31323           NGC  3516                   NGC  3516         
250  32077           NGC  4593                   NGC  4593         
251  32153           UGC  7064                   UGC  7064         
252  32250           NGC  5135                   NGC  5135         
253  32442           Mrk  231                    Mrk  231          
254  32555           Mrk   79                    Mrk   79          
255  32613         LEDA   68751                LEDA   68751        
256  32683           NGC  7319                   NGC  7319         
257  32780           Mrk  863                   

In [32]:
NeIR_obj_t

Main_Id,Id_used_NED,RA,DEC,otype_txt,redshift,rvz_type,otypes,cds_url,SMB_sample_index
object,object,float64,float64,object,float64,str1,object,str93,float64
3C 234.0,3C 234.0,150.45634473807,28.7858176354,Sy1,0.18494,z,AGN|G |IR |QSO|Rad|Sy1|X |blu|gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=150.45634473807%2028.7858176354&-c.rs=5,283.0
M 84,M 84,186.2655972083333,12.886983138888887,Sy2,0.00327,z,* |AGN|G |GiC|GiG|IR |Rad|Sy2|UV |X |smm,http://vizier.u-strasbg.fr/viz-bin/sed?-c=186.2655972083333%2012.886983138888887&-c.rs=5,414.0
2MASS J13000535+1632148,NVSS J130005+163212,195.02228295163,16.53743305218,Sy1,0.07993,z,AGN|G |IR |Rad|Sy1|X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=195.02228295163%2016.53743305218&-c.rs=5,428.0
NGC 262,NGC 262,12.196426138730002,31.956956641230004,Sy2,0.015260233939595302,v,* |AGN|Bla|G |GiG|IR |QSO|Rad|Sy2|X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=12.196426138730002%2031.956956641230004&-c.rs=5,484.0
IRAS 04385-0828,IRAS 04385-0828,70.2290037068,-8.37281297812,Sy2,0.015064,z,* |AGN|G |GiG|IR |Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=70.2290037068%20-8.37281297812&-c.rs=5,541.0
NGC 4945,NGC 4945,196.3636625,-49.4679,Sy2,0.0017968563040435814,v,AGN|G |GiG|IR |Rad|Sy2|X |cm |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=196.3636625%20-49.4679&-c.rs=5,628.0
IC 4553,IC 4553,233.73843316666665,23.50322455555555,SyG,0.0184,z,AGN|G |GiG|GiP|IG |IR |LIN|Rad|SyG|X |gam|smm,http://vizier.u-strasbg.fr/viz-bin/sed?-c=233.73843316666665%2023.50322455555555&-c.rs=5,654.0
NGC 5252,NGC 5252,204.56612821846997,4.542644796,Sy2,0.02294,z,AG?|AGN|EmG|G |GiG|Sy1|Sy2|X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=204.56612821846997%204.542644796&-c.rs=5,773.0
Mrk 279,Mrk 279,208.26431206837998,69.30816969735,Sy1,0.030601,z,AGN|G |IR |QSO|Sy1|UV |X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=208.26431206837998%2069.30816969735&-c.rs=5,852.0


In [33]:
NeIR_obj_t.write(path_raw+'Obj_Ne-IR_'+today.strftime('%b_%d_%Y')+'.vot',
                    format='votable',overwrite=True)

## Notebook Info

In [18]:
%load_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 8.1.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.13.0-39-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 5.0

astropy: 5.0
pyvo   : 1.2.1
sys    : 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
numpy  : 1.20.3
csv    : 1.0

Watermark: 2.3.0



### Las celdas a continuación son para pruebas:

In [26]:
x=428
simbad_sample_Sy[x]

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes,cds_url
,deg,deg,,,,km / s,,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object,str93
2MASS J13000535+1632148,195.02228295163,16.53743305218,2020yCat.1350....0G,Sy1,2012ApJS..203...21A,23007.57777582905,0.07993,z,45,AGN|G |IR |Rad|Sy1|X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=195.02228295163%2016.53743305218&-c.rs=5


In [27]:
nam=simbad_sample_Sy['main_id'][x]
other_ids = Simbad.query_objectids(nam)
other_ids

ID
bytes29
Gaia EDR3 3937375195178654720
TIC 88137340
2MASS J13000535+1632148
NVSS J130005+163212
2MASX J13000533+1632151
2MASSI J1300053+163214
2MASS J13000534+1632149
FIRST J130005.3+163214
[VV2003c] J130005.3+163214


In [30]:
for gal_id in other_ids['ID']:
    try:
        ned_tab= Ned.get_table(gal_id, table='photometry')
        print('si ',gal_id)
        break   
    except:
        print('no ',gal_id)
        if gal_id == other_ids['ID'][-1]:
            continue

no  Gaia EDR3 3937375195178654720
no  TIC 88137340
no  2MASS J13000535+1632148
si  NVSS J130005+163212


In [29]:
ned_tab

No.,Observed Passband,Photometry Measurement,Uncertainty,Units,Frequency,Flux Density,Upper limit of uncertainty,Lower limit of uncertainty,Upper limit of Flux Density,Lower limit of Flux Density,NED Uncertainty,NED Units,Refcode,Significance,Published frequency,Frequency Mode,Coordinates Targeted,Spatial Mode,Qualifiers,Comments
,,,,,Hz,Jy,,,,,,,,,,,,,,
int32,object,float64,object,object,float64,float64,float64,float64,float64,float64,object,object,object,object,object,object,object,object,object,object
1,14-195 keV (Swift),1.44e-11,,erg/cm^2^/s,2.53e+19,5.69e-08,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,104.50 keV,Broad-band measurement,,Modelled datum,Observed flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
2,14-195 keV (Swift),1.27e-11,,erg/cm^2^/s,2.53e+19,5.02e-08,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,104.50 keV,Broad-band measurement,,Modelled datum,Intrinsic flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
3,14-195 keV (Swift),1.144e-11,+/-8.42E-12,erg/s/cm^2^,2.53e+19,4.52e-08,3.33e-08,3.33e-08,--,--,+/-3.33E-08,Jy,2018ApJS..235....4O,90% confidence,104.50 keV,Broad-band measurement,195.03084850 +16.57591628 (J2000),From fitting to map,S/N = 6.10 ; power law model,From new raw data; NED frequency assigned to mid-point ofband in keV
4,14-150 keV (Swift),1.11e-11,,erg/cm^2^/s,1.98e+19,5.61e-08,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,82.00 keV,Broad-band measurement,,Modelled datum,Intrinsic flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
5,20-50 keV (Swift),3.6e-12,,erg/cm^2^/s,8.46e+18,4.26e-08,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,35.00 keV,Broad-band measurement,,Modelled datum,Intrinsic flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
6,2-10 keV (Swift),2.1e-12,,erg/cm^2^/s,1.45e+18,1.45e-07,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,6.00 keV,Broad-band measurement,,Modelled datum,Observed flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
7,2-10 keV (Swift),2.2e-12,,erg/cm^2^/s,1.45e+18,1.52e-07,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,6.00 keV,Broad-band measurement,,Modelled datum,Intrinsic flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
8,0.3-8 keV (Chandra),5.7e-12,,erg/cm^2^/s,1e+18,5.7e-07,--,--,--,--,,Jy,2016ApJS..224...40W,no uncertainty reported,4.15 keV,Broad-band measurement,,Flux integrated from map,Average flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
